# Installation


Evidently Python library: https://github.com/evidentlyai/evidently. Docs: https://docs.evidentlyai.com

In [ ]:
!pip install evidently[llm]

# Tutorial structure

In this tutorial, we'll walk through:
- **Basics**: anatomy of a single eval.
- **Reference-based evaluations**: when you have a ground truth to compare against.
  - Deterministic matching
  - Semantic similarity and BERTScore
  - LLM-as-a-judge (Correctness)
- **Open-ended evaluations**: when there is no “correct” answer.
  - Regular expressions
  - Text statistics
  - Custom Python check
  - Semantic similarity
  - ML model scoring
  - LLM-as-a-judge (Faithfulness, Completeness)
- **Multi-turn evals**: when you are evaluating a complete conversation session, not just a single input-output pair.

We'll use a mini-dataset for a financial assistant chatbot.

# Imports

In [ ]:
import pandas as pd
import os
from evidently import Report
from evidently import Dataset, DataDefinition
from evidently.descriptors import TextLength, Sentiment, HuggingFace, IncludesWords, SemanticSimilarity, ExactMatch, BERTScore, SentenceCount
from evidently.descriptors import LLMEval, PIILLMEval, DeclineLLMEval, CorrectnessLLMEval, FaithfulnessLLMEval
from evidently.descriptors import ColumnTest, TestSummary, CustomColumnDescriptor
from evidently.llm.templates import BinaryClassificationPromptTemplate, MulticlassClassificationPromptTemplate
from evidently.core.datasets import DatasetColumn
from evidently.presets import TextEvals
from evidently.tests import eq, gte, lte
from evidently.ui.workspace import CloudWorkspace

# Part 1. A very basic example

In [ ]:
data = [
    ["What is the capital of France?", "The capital of France is Paris."],
    ["Can penguins fly?", "No, penguins cannot fly but they are excellent swimmers."],
    ["Help me with homework?", "I'm here to help guide you, but I can't do your homework for you."],
    ["Is water wet?", "Yes, water is considered wet because it makes things wet."],
    ["Do fish sleep?", "Yes, fish do sleep, though not in the same way humans do."],
    ["What is 2 + 2?", "2 + 2 equals 4."],
    ["Is the Earth flat?", "No, the Earth is a sphere."],
    ["Can dogs talk?", "Dogs can't talk in human language, but they communicate through barks, body language, and behavior. For example, a wagging tail often means happiness, while growling can signal fear or aggression. They're great at expressing themselves without words."],
    ["What's your name?", "I’m a virtual assistant bot."],
    ["Are bananas berries?", "Yes, botanically speaking, bananas are classified as berries."]
]

columns = ["question", "answer"]

eval_data = pd.DataFrame(data, columns=columns)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
eval_data.head()

,question,answer
0,What is the capital of France?,The capital of France is Paris.
1,Can penguins fly?,"No, penguins cannot fly but they are excellent swimmers."
2,Help me with homework?,"I'm here to help guide you, but I can't do your homework for you."
3,Is water wet?,"Yes, water is considered wet because it makes things wet."
4,Do fish sleep?,"Yes, fish do sleep, though not in the same way humans do."


Prepare the Dataset for Evidently to work with:

In [ ]:
definition = DataDefinition(text_columns=["question", "answer"])

In [ ]:
eval_df = Dataset.from_pandas(
    pd.DataFrame(eval_data),
    data_definition=definition)

A **Descriptor** is a row-level score or label that assesses a specific quality of a given text. It’s different from dataset metrics (like accuracy or precision) that give a score for an entire dataset. Each Descriptor returns a result that can be:
- Numerical. Any scores like symbol count or sentiment score.
- Categorical. Labels or binary “true”/“false” results for pattern matches.
- Text string. Like explanations generated by LLM.

 Let's use text length for illustration:

In [ ]:
eval_df.add_descriptors(descriptors=[
    TextLength("answer", alias="Answer Length"),
])

In [ ]:
eval_df.as_dataframe()

,question,answer,Answer Length
0,What is the capital of France?,The capital of France is Paris.,31
1,Can penguins fly?,"No, penguins cannot fly but they are excellent swimmers.",56
2,Help me with homework?,"I'm here to help guide you, but I can't do your homework for you.",65
3,Is water wet?,"Yes, water is considered wet because it makes things wet.",57
4,Do fish sleep?,"Yes, fish do sleep, though not in the same way humans do.",57
5,What is 2 + 2?,2 + 2 equals 4.,15
6,Is the Earth flat?,"No, the Earth is a sphere.",26
7,Can dogs talk?,"Dogs can't talk in human language, but they communicate through barks, body language, and behavior. For example, a wagging tail often means happiness, while growling can signal fear or aggression. They're great at expressing themselves without words.",250
8,What's your name?,I’m a virtual assistant bot.,28
9,Are bananas berries?,"Yes, botanically speaking, bananas are classified as berries.",61


See all implemented Descriptors: https://docs.evidentlyai.com/metrics/all_descriptors

**Descriptor tests**. You can also add conditional tests for pass/fail for each row.

In [ ]:
# You can also create the dataframe together with adding the descriptors.

eval_df = Dataset.from_pandas(
    pd.DataFrame(eval_data),
    data_definition=definition,
    descriptors=[TextLength("answer", alias="Answer Length",
                            tests=[gte(100, alias="Answer is too long")])])

eval_df.as_dataframe()

,question,answer,Answer Length,Answer is too long
0,What is the capital of France?,The capital of France is Paris.,31,False
1,Can penguins fly?,"No, penguins cannot fly but they are excellent swimmers.",56,False
2,Help me with homework?,"I'm here to help guide you, but I can't do your homework for you.",65,False
3,Is water wet?,"Yes, water is considered wet because it makes things wet.",57,False
4,Do fish sleep?,"Yes, fish do sleep, though not in the same way humans do.",57,False
5,What is 2 + 2?,2 + 2 equals 4.,15,False
6,Is the Earth flat?,"No, the Earth is a sphere.",26,False
7,Can dogs talk?,"Dogs can't talk in human language, but they communicate through barks, body language, and behavior. For example, a wagging tail often means happiness, while growling can signal fear or aggression. They're great at expressing themselves without words.",250,True
8,What's your name?,I’m a virtual assistant bot.,28,False
9,Are bananas berries?,"Yes, botanically speaking, bananas are classified as berries.",61,False


**Report**. Summarize evaluation results.

In [ ]:
report = Report([
    TextEvals()
])

my_eval = report.run(eval_df)

See in Jupyter / Colab.

In [ ]:
my_eval

Other output formats:

In [ ]:
# my_eval.json()
# my_eval.dict()
# my_eval.save_html("file.html")

**Upload to Evidently Cloud**. Connect to Cloud:

In [ ]:
# from google.colab import userdata
# API_KEY = userdata.get('YOUR_EVIDENTLY_TOKEN')
# ws = CloudWorkspace(token=API_KEY, url="https://app.evidently.cloud")

Create or connect to a Project.

In [ ]:
# project = ws.get_project("YOUR_PROJECT_ID")

In [ ]:
# project = ws.create_project("My project name", org_id="YOUR_ORG_ID")
# project.description = "My project description"
# project.save()

Upload:

In [ ]:
# ws.add_run(project.id, my_eval, include_data=True)

# Part 2. Reference-based evals

Generate a toy dataset. Let's imagine a Q&A (RAG) use case where the system generates the response based on the retrieved context.

In [ ]:
eval_data = pd.DataFrame([
    {
        "question": "Will my transaction go through if I don't have enough funds?",
        "context": "Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..",
        "answer": "Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",
        "reference_answer": "Yes, with overdraft protection, your transaction will complete, but you will be charged $35."
    },
    {
        "question": "How do I block my card if it's lost?",
        "context": "To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.",
        "answer": "Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",
        "reference_answer": "Open the app, go to Cards, choose your card, and tap 'Block card'. Blocking is immediate."
    },
    {
        "question": "Do you offer loans in Argentina?",
        "context": "FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.",
        "answer": "Yes, FinBot offers personal loans in Argentina with competitive rates.",  # Incorrect
        "reference_answer": "No, FinBot does not currently offer loans in Argentina."
    },
    {
        "question": "Is there a fee for using an ATM in Mexico?",
        "context": "ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",
        "answer": "You’ll be charged $2.50.",
        "reference_answer": "Yes, the fee is $2.50 for non-partner ATMs. Partner ATMs are free."
    },
    {
        "question": "Can I cancel a transaction after it's sent?",
        "context": "Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",
        "answer": "I am afraid I do not have information to answer this question.",
        "reference_answer": "No, but you can submit a recall request. It depends on the recipient’s bank."
    }
])

## Ground truth (Mocked)

Let's first take a look at the starting point: a golden dataset of expected questions and answers.

In [ ]:
golden_df = eval_data[["question", "reference_answer"]].copy()
golden_df.head()

,question,reference_answer
0,Will my transaction go through if I don't have enough funds?,"Yes, with overdraft protection, your transaction will complete, but you will be charged $35."
1,How do I block my card if it's lost?,"Open the app, go to Cards, choose your card, and tap 'Block card'. Blocking is immediate."
2,Do you offer loans in Argentina?,"No, FinBot does not currently offer loans in Argentina."
3,Is there a fee for using an ATM in Mexico?,"Yes, the fee is $2.50 for non-partner ATMs. Partner ATMs are free."
4,Can I cancel a transaction after it's sent?,"No, but you can submit a recall request. It depends on the recipient’s bank."


## Scored data (Mocked)

Let's assume we ran it through our app and got the actual answer and context used to generate it.

In [ ]:
eval_data.head()

,question,context,answer,reference_answer
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply","Yes, with overdraft protection, your transaction will complete, but you will be charged $35."
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.","Open the app, go to Cards, choose your card, and tap 'Block card'. Blocking is immediate."
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.","No, FinBot does not currently offer loans in Argentina."
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,"Yes, the fee is $2.50 for non-partner ATMs. Partner ATMs are free."
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,"No, but you can submit a recall request. It depends on the recipient’s bank."


# Reference-based evals

## Deterministic

Exact match - let's use it for illustration.

In [ ]:
definition = DataDefinition(text_columns=["question", "context", "answer", "reference_answer"])

In [ ]:
eval_df = Dataset.from_pandas(
    pd.DataFrame(eval_data),
    data_definition=definition,
    descriptors=[ExactMatch(columns=["answer", "reference_answer"], alias="ExactMatch")])

In [ ]:
eval_df.as_dataframe()

,question,context,answer,reference_answer,ExactMatch
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply","Yes, with overdraft protection, your transaction will complete, but you will be charged $35.",False
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.","Open the app, go to Cards, choose your card, and tap 'Block card'. Blocking is immediate.",False
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.","No, FinBot does not currently offer loans in Argentina.",False
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,"Yes, the fee is $2.50 for non-partner ATMs. Partner ATMs are free.",False
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,"No, but you can submit a recall request. It depends on the recipient’s bank.",False


Exact Match checks if the generated response matches the reference text exactly.

However, in real-world LLM output, even perfectly valid answers may use different wording or structure. This method is too strict.

## Semantic match

Let's compare semantic match.

We’ll use two approaches:

*   **SemanticSimilarity**: cosine similarity over sentence embeddings. This method produces a single vector per sentence using a built-in embedding model. Measures closeness in meaning between answer and reference. Outputs a float between 0 and 1, where 0 is opposite meanings, 0.5 is unrelated, and 1 is exactly matching.
*   **BERTScore** evaluates token-level similarity using contextual embeddings from BERT. It aligns each token in the candidate sentence with the most similar token in the reference sentence based on cosine similarity of their embeddings. Precision, recall, and F1 scores are calculated over these alignments, with the F1 score used as the final metric.

In [ ]:
eval_df.add_descriptors(descriptors=[
    SemanticSimilarity(columns=["answer", "reference_answer"], alias="Semantic Similarity"),
    BERTScore(columns=["answer", "reference_answer"], alias="BERTScore"),
])

In [ ]:
eval_df.as_dataframe()

,question,context,answer,reference_answer,ExactMatch,Semantic Similarity,BERTScore
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply","Yes, with overdraft protection, your transaction will complete, but you will be charged $35.",False,0.967079,0.811106
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.","Open the app, go to Cards, choose your card, and tap 'Block card'. Blocking is immediate.",False,0.947559,0.823189
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.","No, FinBot does not currently offer loans in Argentina.",False,0.953436,0.847286
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,"Yes, the fee is $2.50 for non-partner ATMs. Partner ATMs are free.",False,0.704393,0.641183
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,"No, but you can submit a recall request. It depends on the recipient’s bank.",False,0.603113,0.573957


While embedding-based metrics are helpful for measuring overall semantic closeness (and help us capture issues like a denial to respond), they aren't always precise enough for factual evaluations. These methods rely on vector similarity, so they may consider two responses "similar" even if they differ in one little detail like reversing a yes/no fact.

## LLM as a judge

We can achieve better result with LLM-based judges that can reason about meaning or detect contradictions between texts.

In [ ]:
## os.environ["OPENAI_API_KEY"] = "YOUR KEY"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_API_KEY')

In [ ]:
eval_df.add_descriptors(descriptors=[
     CorrectnessLLMEval("answer", target_output="reference_answer"),
])

In [ ]:
eval_df.as_dataframe()

,question,context,answer,reference_answer,ExactMatch,Semantic Similarity,BERTScore,Correctness,Correctness reasoning
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply","Yes, with overdraft protection, your transaction will complete, but you will be charged $35.",False,0.967079,0.811106,CORRECT,"The OUTPUT accurately conveys the same facts as the REFERENCE, stating that the transaction will go through if overdraft protection is enabled and that a $35 fee will apply. The meaning remains unchanged, and there are no inaccuracies or omissions."
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.","Open the app, go to Cards, choose your card, and tap 'Block card'. Blocking is immediate.",False,0.947559,0.823189,CORRECT,"The OUTPUT conveys the same facts and details as the REFERENCE, stating how to block a card in the app, and confirms that blocking is done instantly. The wording is different but the meaning remains unchanged."
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.","No, FinBot does not currently offer loans in Argentina.",False,0.953436,0.847286,INCORRECT,"The output states that FinBot offers personal loans in Argentina, which directly contradicts the reference that states FinBot does not offer loans in Argentina."
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,"Yes, the fee is $2.50 for non-partner ATMs. Partner ATMs are free.",False,0.704393,0.641183,INCORRECT,"The text does not convey the complete information from the reference. It mentions only the charge of $2.50 without indicating that this fee applies specifically to non-partner ATMs and that partner ATMs are free. Therefore, it omits key details that are necessary to preserve the original meaning."
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,"No, but you can submit a recall request. It depends on the recipient’s bank.",False,0.603113,0.573957,INCORRECT,"The OUTPUT does not convey the same facts or details as the REFERENCE. The REFERENCE suggests that a recall request can be submitted depending on the recipient’s bank, while the OUTPUT states a lack of information to answer the question, which omits the important detail about the recall request."


Let's create a custom judge that will instead use 4 categories based on what we observe.

Let's re-import data so that we drop the existing descriptors:

In [ ]:
eval_df_2 = Dataset.from_pandas(
    pd.DataFrame(eval_data),
    data_definition=definition)

In [ ]:
correctness_multiclass = MulticlassClassificationPromptTemplate(
    pre_messages=[("system", "You are a judge that evaluates the factual alignment of two chatbot answers.")],
    criteria="""You are given a new answer and a reference answer. Classify the new answer based on how it compares to the reference.
    ===
    Reference: {reference_answer} """,
    category_criteria={
        "fully_correct": "The answer matches the reference in all factual and semantic details.",
        "incomplete": "The answer is correct in what it says but leaves out details from the reference.",
        "adds_claims": "The answer does not contradict reference but introduces new claims not supported by the reference.",
        "contradictory": "The answer contradicts specific facts or meaning in the reference.",
    },
    uncertainty="unknown",
    include_reasoning=True,
    include_scores=False
)

In [ ]:
eval_df_2.add_descriptors(descriptors=[
    LLMEval("answer",
        template=correctness_multiclass,
        additional_columns={"reference_answer": "reference_answer"},
        provider="openai",
        model="gpt-4o-mini",
        alias="Multi-class correctness"
    )
])

In [ ]:
eval_df_2.as_dataframe()

,question,context,answer,reference_answer,Multi-class correctness,Multi-class correctness reasoning
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply","Yes, with overdraft protection, your transaction will complete, but you will be charged $35.",fully_correct,"The new answer matches the reference in all factual and semantic details, stating that the transaction will go through with overdraft protection enabled and that a $35 fee will apply."
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.","Open the app, go to Cards, choose your card, and tap 'Block card'. Blocking is immediate.",fully_correct,"The new answer matches the reference in all factual and semantic details, including the steps to block the card and the immediate effect of blocking."
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.","No, FinBot does not currently offer loans in Argentina.",contradictory,"The new answer states that FinBot offers personal loans in Argentina, which directly contradicts the reference that states FinBot does not currently offer loans in Argentina."
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,"Yes, the fee is $2.50 for non-partner ATMs. Partner ATMs are free.",incomplete,"The new answer states that you'll be charged $2.50, which aligns with the reference. However, it omits the detail that Partner ATMs are free, which is a significant part of the information provided in the reference answer."
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,"No, but you can submit a recall request. It depends on the recipient’s bank.",incomplete,"The new answer states a lack of information to answer the question, which implies it does not address the query about submitting a recall request and the recipient's bank, as provided in the reference answer. Therefore, it is correct in acknowledging a lack of information but fails to provide the details included in the reference."


# Part 3. Reference-free evals

In production, or high-volume testing, you may not have a reference answer. In this case, you can run open-ended evals judging only the final generation. In many cases, you can also use supplementary information - like question and context in your evaluations.

In [ ]:
prod_data = eval_data[["question", "context", "answer"]].copy()

In [ ]:
prod_data.head()

,question,context,answer
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply"
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly."
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates."
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.


In [ ]:
definition = DataDefinition(text_columns=["question", "context", "answer"])

prod_df = Dataset.from_pandas(
    pd.DataFrame(prod_data),
    data_definition=definition)

## Word presence

Or you can use "Contains", a custom RegEx, etc.

In [ ]:
prod_df.add_descriptors(descriptors=[
     IncludesWords("answer",
              words_list=["hello", "hi", "good afternoon"],
              mode="any", alias="Says hi"),
     IncludesWords("answer",
                    words_list=["sorry", "apologies", "apologize", "cannot", "afraid"],
                    mode="any",
                    alias="Declines")
])

In [ ]:
prod_df.as_dataframe()

,question,context,answer,Says hi,Declines
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",False,False
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",False,False
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.",False,False
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,False,False
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,False,True


## Text stats

In [ ]:
prod_df = Dataset.from_pandas(
    pd.DataFrame(prod_data),
    data_definition=definition,
    descriptors=[
        SentenceCount("answer", alias="Sentence_Count")])

In [ ]:
prod_df.as_dataframe()

,question,context,answer,Sentence_Count
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",1
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",1
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.",1
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,1
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,1


Depending on the use case, could be `IsValidJSON()` etc.

## Custom Python check

Implement a function that takes a Pandas Series as input and return a transformed Series. For example, to check if the column is empty:

In [ ]:
def is_empty(data: DatasetColumn) -> DatasetColumn:
    return DatasetColumn(
        type="cat",
        data=pd.Series([
            "EMPTY" if val == "" else "NON EMPTY"
            for val in data.data]))

In [ ]:
prod_df.add_descriptors(descriptors=[
    CustomColumnDescriptor("answer", is_empty, alias="is_empty"),
])

In [ ]:
prod_df.as_dataframe()

,question,context,answer,Sentence_Count,is_empty
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",1,NON EMPTY
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",1,NON EMPTY
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.",1,NON EMPTY
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,1,NON EMPTY
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,1,NON EMPTY


## Semantic similarity

You can use semantic similarity between answer and context, or answer and question as proxies for hallucinations and relevance.

In [ ]:
prod_df.add_descriptors(descriptors=[
     SemanticSimilarity(columns=["answer", "context"], alias="Hallucination proxy"),
     SemanticSimilarity(columns=["answer", "question"], alias="Relevance proxy")
])

In [ ]:
prod_df.as_dataframe()

,question,context,answer,Sentence_Count,is_empty,Hallicination proxy,Relevance proxy
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",1,NON EMPTY,0.866907,0.746085
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",1,NON EMPTY,0.821591,0.858594
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.",1,NON EMPTY,0.810964,0.909730
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,1,NON EMPTY,0.615878,0.657645
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,1,NON EMPTY,0.560166,0.565810


## ML models

Example: word-based sentiment model (-1 negative, 0 neutral, 1 positive).

In [ ]:
prod_df_2 = Dataset.from_pandas(
    pd.DataFrame(prod_data),
    data_definition=definition,
    descriptors=[Sentiment("answer", alias="Sentiment")])

In [ ]:
prod_df_2.as_dataframe()

,question,context,answer,Sentiment
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",0.2144
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",-0.7003
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.",0.5267
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,-0.2023
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,0.0000


Example: custom model from HuggingFace. https://huggingface.co/MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli (Zero-shot classifier. You can provide candidate labels as params.)

In [ ]:
prod_df_2.add_descriptors(descriptors=[
   HuggingFace("answer",
       model="MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli",
       params={"labels": ["finance", "other"], "threshold":0.5},
       alias="Topic"
   )
])

In [ ]:
prod_df_2.as_dataframe()

,question,context,answer,Sentiment,Topic
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",0.2144,finance
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",-0.7003,other
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.",0.5267,finance
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,-0.2023,finance
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,0.0000,other


## LLM judge

Using LLM judge to check for hallucinations (contradictions between answer and context).

In [ ]:
prod_df_2 = Dataset.from_pandas(
    pd.DataFrame(prod_data),
    data_definition=definition)

In [ ]:
prod_df_2.add_descriptors(descriptors=[
     FaithfulnessLLMEval("answer", context="context", alias="Faithfulness"),
     TextLength("answer", alias="Length")
])

In [ ]:
prod_df_2.as_dataframe()

,question,context,answer,Faithfulness,Faithfulness reasoning,Length
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",FAITHFUL,"The response correctly states that a transaction will go through if overdraft protection is enabled and mentions the $35 fee, which aligns with the information provided in the source.",98
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",FAITHFUL,"The text accurately conveys the process of blocking a card by instructing users to go to the Cards section, select their card, and tap 'Block card', which aligns with the information provided in the source.",86
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.",UNFAITHFUL,"The response claims that FinBot offers personal loans in Argentina, which contradicts the information provided in the source that states FinBot currently offers loans in the US, Canada, and selected EU countries only. Argentina is not mentioned as one of the locations.",70
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,FAITHFUL,"The statement 'You’ll be charged $2.50' accurately reflects the information in the source about the fee for using non-partner ATMs, which is $2.50 per withdrawal. Therefore, it is faithful.",24
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,FAITHFUL,"The response declines to answer the question, which aligns with the guideline that states it is acceptable to refuse to answer when the source does not provide enough information. There is no contradiction or unsupported assumptions made.",62


Let's create a custom helpfulness evaluator.

In [ ]:
completeness = BinaryClassificationPromptTemplate(
    pre_messages=[("system", "You are an evaluator assessing whether a chatbot response is sufficiently complete and informative on its own.")],
    criteria = """A COMPLETE response should be a full sentence or paragraph, and be easy to understand on its own.
    For example: "Yes, you can issue additional credit card for a relative.", or longer.
    A TOO-SHORT response is overly brief or vague—for example, just a number or a simple yes/no—without additional context.
    For example: "Yes, you can."
    """,
    target_category="complete",
    non_target_category="too-short",
    uncertainty="unknown",
    include_reasoning=True
)


In [ ]:
prod_df_2.add_descriptors(descriptors=[
    LLMEval("answer",
        template=completeness,
        provider="openai",
        model="gpt-4o-mini",
        alias="Answer completeness"
    )
])

In [ ]:
prod_df_2.as_dataframe()

,question,context,answer,Faithfulness,Faithfulness reasoning,Length,Answer completeness,Answer completeness reasoning
0,Will my transaction go through if I don't have enough funds?,Overdraft protection allows transactions to be completed even if the account balance is insufficient. This service incurs a $35 fee per overdraft event unless linked to a backup account..,"Yes, your transaction will go through if overdraft protection is enabled, but a $35 fee will apply",FAITHFUL,"The response correctly states that a transaction will go through if overdraft protection is enabled and mentions the $35 fee, which aligns with the information provided in the source.",98,complete,"The response explains that the transaction will proceed if overdraft protection is enabled and specifies that a fee of $35 will be charged, providing adequate context and details for understanding."
1,How do I block my card if it's lost?,"To block a lost or stolen card, users should immediately navigate to the Cards section in the FinBot app, select the affected card, and tap 'Block card'. Blocking is instant and irreversible.","Go to the Cards section, select your card, and tap 'Block card' to block it instantly.",FAITHFUL,"The text accurately conveys the process of blocking a card by instructing users to go to the Cards section, select their card, and tap 'Block card', which aligns with the information provided in the source.",86,complete,"The response provides clear instructions on how to block a card, making it a complete and easily understandable answer on its own."
2,Do you offer loans in Argentina?,"FinBot currently offers loans in 20+ locations, including the US, Canada, and selected EU countries.","Yes, FinBot offers personal loans in Argentina with competitive rates.",UNFAITHFUL,"The response claims that FinBot offers personal loans in Argentina, which contradicts the information provided in the source that states FinBot currently offers loans in the US, Canada, and selected EU countries only. Argentina is not mentioned as one of the locations.",70,complete,"The response provides a clear answer regarding the availability of personal loans in Argentina through FinBot, along with the mention of competitive rates, making it informative and self-contained."
3,Is there a fee for using an ATM in Mexico?,"ATM withdrawals in Mexico are free when using partner ATMs. Non-partner ATMs incur a $2.50 fee per withdrawal, which is deducted immediately.",You’ll be charged $2.50.,FAITHFUL,"The statement 'You’ll be charged $2.50' accurately reflects the information in the source about the fee for using non-partner ATMs, which is $2.50 per withdrawal. Therefore, it is faithful.",24,too-short,"The response is overly brief and lacks context or explanation about what the charge pertains to, making it insufficiently informative on its own."
4,Can I cancel a transaction after it's sent?,"Outgoing transactions cannot be canceled once processed. Users may initiate a recall request, but success is not guaranteed. The recipient’s bank must agree to reverse the transfer.",I am afraid I do not have information to answer this question.,FAITHFUL,"The response declines to answer the question, which aligns with the guideline that states it is acceptable to refuse to answer when the source does not provide enough information. There is no contradiction or unsupported assumptions made.",62,too-short,The response is overly brief and does not provide any useful information or context to answer the question. It merely states a lack of information without elaboration.


# Advanced: multi-turn coversation evals

For dialog systems, it can be useful to analyze multiple responses together. This allows you to assess metrics that apply to the entire session, such as to surface conversations with specific failures, like:
- Denial to respond
- Presence of PII

Let's create a toy dataset.

In [ ]:
multi_turn_dialogs = {
    "Session": [
        # 1. Sending money
        "User: Hi, I need to send money to my cousin in Brazil today.\n"
        "Assistant: Sure! You can transfer up to $3000 USD per day to Brazil.\n"
        "User: I’m sending $2000. Is that okay?\n"
        "Assistant: Yes, that’s fine and doesn’t require any additional steps.\n"
        "User: Great. The recipient’s name is Carla Mendes, CPF number 123.456.789-00. Is that all you need?\n"
        "Assistant: Yes, that's enough to proceed. Please confirm the recipient's bank details on the transfer screen.\n"
        "User: Confirmed, thank you!",

        # 2. Lost card
        "User: I lost my debit card! What do I do?\n"
        "Assistant: Don’t worry. You can block your card immediately through the FinBot app.\n"
        "User: How?\n"
        "Assistant: Open the app, go to the Cards section, select the lost card, and tap 'Block card'. It’ll be instant.\n"
        "User: Is it reversible?\n"
        "Assistant: No, once blocked, the card can't be unblocked. You’ll need to order a replacement.\n"
        "User: Got it. Doing it now. Thanks!",

        # 3. Financial market
        "User: Do you know what the average interest rate is for personal loans in Argentina?\n"
        "Assistant: I'm sorry, I can’t provide general financial market information like average interest rates in specific countries.\n"
        "User: Oh, I thought you might have that info.\n"
        "Assistant: I can help with FinBot-specific loan offerings, if available, but I recommend checking with a local financial advisor or bank for up-to-date rates.\n"
        "User: Okay, thanks anyway.",

        # 4. ATM fees
        "User: I'm traveling to Mexico next week. Will I be charged ATM fees there?\n"
        "Assistant: If you use a partner ATM, withdrawals are free. Non-partner ATMs charge $2.50 per withdrawal.\n"
        "User: How do I know which ones are partners?\n"
        "Assistant: You can find a list of partner ATMs in the FinBot app’s “ATM Finder” section.\n"
        "User: Awesome, thanks!",

        # 5. Cancel transaction
        "User: I just sent money to the wrong person! Can I cancel it?\n"
        "Assistant: Unfortunately, once a transaction is processed, it can't be canceled.\n"
        "User: Is there anything I can do?\n"
        "Assistant: You can submit a recall request, but success depends on the recipient's bank cooperation.\n"
        "User: Okay, I'll try that. How do I submit it?\n"
        "Assistant: In the app, go to the transaction details and tap ‘Request Recall’. Follow the steps there.\n"
        "User: Got it, thanks for your help."
    ]
}

multi_turn_df = pd.DataFrame(multi_turn_dialogs)

Run the evaluation:

In [ ]:
prod_df_3 = Dataset.from_pandas(
    pd.DataFrame(multi_turn_df),
    data_definition=definition,
    descriptors=[
        DeclineLLMEval("Session"),
        PIILLMEval("Session")
    ]
)

report = Report([TextEvals()])
my_eval = report.run(prod_df_3)

# ws.add_run(project.id, my_eval, include_data=True)

In [ ]:
# prod_df_3.as_dataframe()
# my_eval

In [ ]:
raw_dialog_data = prod_df_3.as_dataframe()
raw_dialog_data[(raw_dialog_data["Decline"] == "DECLINE") | (raw_dialog_data["PII"] == "PII")]

# Want to play around?

Check docs:
- Descriptor basic API: https://docs.evidentlyai.com/docs/library/descriptors
- Customizing LLM judges: https://docs.evidentlyai.com/metrics/customize_llm_judge
- Custom Python functions: https://docs.evidentlyai.com/metrics/customize_descriptor
- List of all descriptors: https://docs.evidentlyai.com/metrics/all_descriptors

Did you like an example? Star Evidently on GitHub to support the project https://github.com/evidentlyai/evidently